In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import root_mean_squared_error

In [103]:
# Dataset used = kaggle - Regression used car price

In [51]:
train = pd.read_csv("carPriseTrain.csv")
test = pd.read_csv("carPriseTest.csv")

In [53]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [19]:
train.nunique()


id              188533
brand               57
model             1897
model_year          34
milage            6651
fuel_type            7
engine            1117
transmission        52
ext_col            319
int_col            156
accident             2
clean_title          1
price             1569
dtype: int64

In [33]:
train.isna().sum()

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type           0
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [23]:
train['fuel_type'].unique()

array(['Gasoline', 'E85 Flex Fuel', nan, 'Hybrid', 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [29]:
train['fuel_type'] = train.fuel_type.fillna("unknown")

In [25]:
train["brand"].value_counts()

brand
Ford             23088
Mercedes-Benz    19172
BMW              17028
Chevrolet        16335
Audi             10887
Porsche          10612
Land              9525
Toyota            8850
Lexus             8643
Jeep              6474
Cadillac          4674
RAM               4249
Nissan            3930
Tesla             3738
INFINITI          3276
GMC               3215
Dodge             3133
Mazda             2719
Kia               2497
Lincoln           2423
Subaru            2381
Acura             2282
Honda             2101
Hyundai           2045
Volkswagen        1765
Jaguar            1319
Bentley           1155
MINI              1064
Genesis            969
Buick              940
Maserati           939
Lamborghini        809
Chrysler           727
Volvo              723
Alfa               682
Rivian             590
Rolls-Royce        561
Mitsubishi         551
Pontiac            538
Hummer             520
Ferrari            359
McLaren            243
Aston              238
Satur

In [31]:
train["fuel_type"].value_counts()

fuel_type
Gasoline          165940
Hybrid              6832
E85 Flex Fuel       5406
unknown             5083
Diesel              3955
–                    781
Plug-In Hybrid       521
not supported         15
Name: count, dtype: int64

In [37]:
ohe = OneHotEncoder( 
    handle_unknown = 'infrequent_if_exist',
    min_frequency= 3000,
    sparse_output = False
).set_output(transform = 'pandas')




In [55]:
x_train = train.drop('price', axis = 1)
y_train = train['price']

In [57]:
ohe = OneHotEncoder(handle_unknown = 'infrequent_if_exist',
                    sparse_output = False, 
                    min_frequency = 3000
                   ).set_output(transform = 'pandas')

ct_ohe = make_column_transformer(("passthrough", make_column_selector(dtype_exclude = object)), 
                                 (ohe, make_column_selector(dtype_include = object)),
                                 verbose_feature_names_out = False
                                ).set_output(transform = 'pandas')

x_train_ohe = ct_ohe.fit_transform(x_train)
                                 

In [59]:
x_train_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 63 columns):
 #   Column                                               Non-Null Count   Dtype  
---  ------                                               --------------   -----  
 0   id                                                   188533 non-null  int64  
 1   model_year                                           188533 non-null  int64  
 2   milage                                               188533 non-null  int64  
 3   brand_Audi                                           188533 non-null  float64
 4   brand_BMW                                            188533 non-null  float64
 5   brand_Cadillac                                       188533 non-null  float64
 6   brand_Chevrolet                                      188533 non-null  float64
 7   brand_Dodge                                          188533 non-null  float64
 8   brand_Ford                                           1

In [61]:
x_test = pd.read_csv("carPriseTest.csv")
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [65]:
x_test_ohe = ct_ohe.transform(x_test)
x_test_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 63 columns):
 #   Column                                               Non-Null Count   Dtype  
---  ------                                               --------------   -----  
 0   id                                                   125690 non-null  int64  
 1   model_year                                           125690 non-null  int64  
 2   milage                                               125690 non-null  int64  
 3   brand_Audi                                           125690 non-null  float64
 4   brand_BMW                                            125690 non-null  float64
 5   brand_Cadillac                                       125690 non-null  float64
 6   brand_Chevrolet                                      125690 non-null  float64
 7   brand_Dodge                                          125690 non-null  float64
 8   brand_Ford                                           1

In [73]:
# Linear Regression
lr = LinearRegression()
lr.fit(x_train_ohe, y_train)
y_pred = lr.predict(x_test_ohe)
y_pred 

array([27632.97567592, 68213.99952729, 66900.33522662, ...,
       29304.22653627, 18413.96061851, 39542.42663029])

In [77]:
y_pred[y_pred < 150] = 150

array([], dtype=float64)

In [81]:
submit = pd.read_csv("sample_submission.csv")
submit

,id,price
0,188533,43878.016
1,188534,43878.016
2,188535,43878.016
3,188536,43878.016
4,188537,43878.016
...,...,...
125685,314218,43878.016
125686,314219,43878.016
125687,314220,43878.016
125688,314221,43878.016


In [83]:
submit['price'] = y_pred
submit

,id,price
0,188533,27632.975676
1,188534,68213.999527
2,188535,66900.335227
3,188536,36927.910789
4,188537,41899.218642
...,...,...
125685,314218,38787.001567
125686,314219,44451.926139
125687,314220,29304.226536
125688,314221,18413.960619


In [85]:
submit.to_csv("sub_1_5_nov.csv", index = False)

# Elastic Net

In [20]:
train = pd.read_csv("carPriseTrain.csv")
test = pd.read_csv("carPriseTest.csv")

In [22]:
x_train = train.drop('price', axis = 1)
y_train = train['price']

In [12]:
ohe = OneHotEncoder(handle_unknown = 'infrequent_if_exist',
                    sparse_output = False, 
                    min_frequency = 3000
                   ).set_output(transform = 'pandas')

ct_ohe = make_column_transformer(("passthrough", make_column_selector(dtype_exclude = object)), 
                                 (ohe, make_column_selector(dtype_include = object)),
                                 verbose_feature_names_out = False
                                ).set_output(transform = 'pandas')

x_train_ohe = ct_ohe.fit_transform(x_train)

In [24]:
# Elastic Net 
# alpha = np.linspace(0.001, 5, 10)
# l1_ratio = np.linspace(0.001, 1, 5)
eln = ElasticNet()

pipe = Pipeline([("CT", ohe), ("ELN", eln)])

kfold = KFold(n_splits=5, 
              random_state=24, 
              shuffle=True)

params = {"ELN__alpha" : np.linspace(0.001, 5, 3),
          "ELN__l1_ratio" : np.linspace(0.001, 1, 3)}

gcv = GridSearchCV(pipe, 
                   param_grid = params,
                   cv = kfold, 
                   verbose = 3)


In [26]:
gcv.fit(x_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\Users\DAI.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.108e+11, tolerance: 9.377e+10
  model = cd_fast.enet_coordinate_descent(


[CV 1/5] END ELN__alpha=0.001, ELN__l1_ratio=0.001;, score=0.091 total time=  10.8s


C:\Users\DAI.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.855e+11, tolerance: 9.096e+10
  model = cd_fast.enet_coordinate_descent(


[CV 2/5] END ELN__alpha=0.001, ELN__l1_ratio=0.001;, score=0.078 total time=  10.6s


...

C:\Users\DAI.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.769e+12, tolerance: 9.775e+10
  model = cd_fast.enet_coordinate_descent(


[CV 4/5] END ELN__alpha=2.5004999999999997, ELN__l1_ratio=1.0;, score=0.112 total time=   8.4s


C:\Users\DAI.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.533e+12, tolerance: 9.187e+10
  model = cd_fast.enet_coordinate_descent(


[CV 5/5] END ELN__alpha=2.5004999999999997, ELN__l1_ratio=1.0;, score=0.086 total time=   8.3s
[CV 1/5] END ELN__alpha=5.0, ELN__l1_ratio=0.001;, score=0.007 total time=   2.4s
[CV 2/5] END ELN__alpha=5.0, ELN__l1_ratio=0.001;, score=0.006 total time=   2.3s
[CV 3/5] END ELN__alpha=5.0, ELN__l1_ratio=0.001;, score=0.007 total time=   2.4s
[CV 4/5] END ELN__alpha=5.0, ELN__l1_ratio=0.001;, score=0.008 total time=   2.3s
[CV 5/5] END ELN__alpha=5.0, ELN__l1_ratio=0.001;, score=0.006 total time=   2.4s
[CV 1/5] END ELN__alpha=5.0, ELN__l1_ratio=0.5005;, score=0.012 total time=   2.5s
[CV 2/5] END ELN__alpha=5.0, ELN__l1_ratio=0.5005;, score=0.010 total time=   2.4s
[CV 3/5] END ELN__alpha=5.0, ELN__l1_ratio=0.5005;, score=0.012 total time=   2.5s
[CV 4/5] END ELN__alpha=5.0, ELN__l1_ratio=0.5005;, score=0.015 total time=   2.4s
[CV 5/5] END ELN__alpha=5.0, ELN__l1_ratio=0.5005;, score=0.011 total time=   2.3s
[CV 1/5] END .ELN__alpha=5.0, ELN__l1_ratio=1.0;, score=0.091 total time=   5.5s

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                      min_frequency=3000,
                                                      sparse_output=False)),
                                       ('ELN', ElasticNet())]),
             param_grid={'ELN__alpha': array([1.0000e-03, 2.5005e+00, 5.0000e+00]),
                         'ELN__l1_ratio': array([0.001 , 0.5005, 1.    ])},
             verbose=3)

In [28]:
y_pred = gcv.predict(test)

In [32]:
np.quantile(y_pred, [0, 0.25, 0.5, 0.75, 1.0])

array([ -6575.45047271,  23901.62058129,  41317.64683113,  60953.85187171,
       133531.35521552])

In [34]:
y_pred[y_pred < 150] = 150

In [42]:
submit = pd.read_csv("sample_submission.csv")
submit.head(3)

,id,price
0,188533,43878.016
1,188534,43878.016
2,188535,43878.016


In [44]:
submit.price = y_pred


0         28003.946074
1         60778.092112
2         72901.668303
3         24353.461933
4         42365.966138
              ...     
125685    38400.208100
125686    36235.022121
125687    18944.121979
125688    18944.121979
125689    37137.116757
Name: price, Length: 125690, dtype: float64

In [46]:
submit.to_csv("sub_2_ElasticNet.csv", index = False)

# KNN

In [57]:
train = pd.read_csv("carPriseTrain.csv")
test = pd.read_csv("carPriseTest.csv")

In [59]:
x_train = train.drop('price', axis = 1)
y_train = train['price']

In [ ]:


# ct_ohe = make_column_transformer(("passthrough", make_column_selector(dtype_exclude = object)), 
#                                  (ohe, make_column_selector(dtype_include = object)),
#                                  verbose_feature_names_out = False
#                                 ).set_output(transform = 'pandas')

# x_train_ohe = ct_ohe.fit_transform(x_train)

In [79]:

ohe = OneHotEncoder(handle_unknown = 'infrequent_if_exist',
                    sparse_output = False, 
                    min_frequency = 3000
                   ).set_output(transform = 'pandas')

knn = KNeighborsRegressor()

scaler_mm = MinMaxScaler()
scaler_std = StandardScaler()

pipe = Pipeline([("CT", ohe), ("SCL", None), ("KNN", knn)])

kfold = KFold(n_splits=5, 
              random_state=24, 
              shuffle=True)

params = {"KNN__n_neighbors" : list(range(1, 8)),
          "SCL" : [scaler_mm, scaler_std, None]}

gcv = GridSearchCV(pipe, 
                   param_grid = params,
                   scoring = 'r2',
                   cv = kfold, 
                   verbose = 3)


In [81]:
gcv.fit(x_train, y_train)

Fitting 5 folds for each of 21 candidates, totalling 105 fits
[CV 1/5] END KNN__n_neighbors=1, SCL=MinMaxScaler();, score=-2.124 total time=  15.9s
[CV 2/5] END KNN__n_neighbors=1, SCL=MinMaxScaler();, score=-2.001 total time=  15.7s
[CV 3/5] END KNN__n_neighbors=1, SCL=MinMaxScaler();, score=-0.553 total time=  16.1s
[CV 4/5] END KNN__n_neighbors=1, SCL=MinMaxScaler();, score=-2.649 total time=  15.7s
[CV 5/5] END KNN__n_neighbors=1, SCL=MinMaxScaler();, score=-1.808 total time=  16.1s
[CV 1/5] END KNN__n_neighbors=1, SCL=StandardScaler();, score=-2.172 total time=  16.0s
[CV 2/5] END KNN__n_neighbors=1, SCL=StandardScaler();, score=-1.976 total time=  16.6s
[CV 3/5] END KNN__n_neighbors=1, SCL=StandardScaler();, score=-0.723 total time=  16.4s
[CV 4/5] END KNN__n_neighbors=1, SCL=StandardScaler();, score=-2.662 total time=  16.5s
[CV 5/5] END KNN__n_neighbors=1, SCL=StandardScaler();, score=-1.830 total time=  16.2s
[CV 1/5] END .....KNN__n_neighbors=1, SCL=None;, score=-2.124 total 

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                      min_frequency=3000,
                                                      sparse_output=False)),
                                       ('SCL', None),
                                       ('KNN', KNeighborsRegressor())]),
             param_grid={'KNN__n_neighbors': [1, 2, 3, 4, 5, 6, 7],
                         'SCL': [MinMaxScaler(), StandardScaler(), None]},
             scoring='r2', verbose=3)

In [93]:
y_pred = gcv.best_estimator_.predict(test)
y_pred



array([18471.42857143, 41378.14285714, 40185.14285714, ...,
       14614.28571429, 14614.28571429, 51957.14285714])

In [ ]:
y_pred[y_pred < 150]

In [101]:
sample = pd.read_csv("sample_submission.csv")
sample.price = y_pred
sample.to_csv("sub_3_KNNRegressor.csv" , index = False)

In [55]:
vars(KNeighborsRegressor())

{'n_neighbors': 5,
 'radius': None,
 'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'p': 2,
 'n_jobs': None,
 'weights': 'uniform'}